In [ ]:
%run ./00_setup_and_config

# Synthetic Accuracy Comparison

**Purpose:** Compare trading strategy performance across synthetic prediction accuracy levels (60%, 70%, 80%, 90%)

**Critical Validation:**
- If 90% accuracy doesn't show advantage → Trading logic has bugs
- Performance should improve monotonically with accuracy
- Identify accuracy threshold where predictions become valuable

**Strategies Tested:** All 9 strategies (4 baselines, 5 prediction-based)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pickle

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("=" * 80)
print("SYNTHETIC ACCURACY COMPARISON ANALYSIS")
print("=" * 80)
print("\nThis notebook validates trading logic by comparing performance")
print("across controlled accuracy levels: 60%, 70%, 80%, 90%")
print("\n⚠️  CRITICAL: If 90% accuracy doesn't beat baselines, logic is broken!")

## Configuration

In [ ]:
# Synthetic model versions (generated by notebook 01)
SYNTHETIC_ACCURACIES = [100, 90, 80, 70, 60]  # Added 100% - CRITICAL TEST
SYNTHETIC_MODELS = [f'synthetic_acc{acc}' for acc in SYNTHETIC_ACCURACIES]

# Strategy groups
BASELINE_STRATEGIES = ['Immediate Sale', 'Equal Batches', 'Price Threshold', 'Moving Average']
PREDICTION_STRATEGIES = ['Consensus', 'Expected Value', 'Risk-Adjusted', 
                        'Price Threshold Predictive', 'Moving Average Predictive']

# Color scheme
ACCURACY_COLORS = {
    100: '#27ae60',  # Dark Green - PERFECT
    90: '#2ecc71',   # Green
    80: '#f1c40f',   # Yellow
    70: '#f39c12',   # Orange
    60: '#e74c3c'    # Red
}

print(f"\n⚠️  CRITICAL: 100% accuracy = PERFECT FORESIGHT")
print(f"   If strategies can't beat baselines with 100% accuracy, algo is BROKEN!")
print(f"\nAccuracy levels to compare: {SYNTHETIC_ACCURACIES}")
print(f"Commodities: {list(COMMODITY_CONFIGS.keys())}")
print(f"Baseline strategies: {len(BASELINE_STRATEGIES)}")
print(f"Prediction strategies: {len(PREDICTION_STRATEGIES)}")

## Load Results for All Accuracy Levels

In [ ]:
# Store all results
all_results = {}

for COMMODITY in COMMODITY_CONFIGS.keys():
    print(f"\n{'='*80}")
    print(f"LOADING RESULTS: {COMMODITY.upper()}")
    print(f"{'='*80}")
    
    all_results[COMMODITY] = {}
    
    for model_version in SYNTHETIC_MODELS:
        accuracy = int(model_version.split('acc')[1])
        
        print(f"\n  Loading {accuracy}% accuracy results...")
        
        try:
            # Get data paths for this model version
            paths = get_data_paths(COMMODITY, model_version)
            
            # Load results from Delta table
            results_df = spark.table(paths['results']).toPandas()
            
            # Load detailed results (for trade-level analysis)
            with open(paths['results_detailed'], 'rb') as f:
                results_detailed = pickle.load(f)
            
            all_results[COMMODITY][accuracy] = {
                'summary': results_df,
                'detailed': results_detailed
            }
            
            print(f"    ✓ Loaded {len(results_df)} strategy results")
            
        except Exception as e:
            print(f"    ⚠️  Could not load {accuracy}% results: {e}")
            print(f"    Note: You may need to run notebook 05 first to generate results")

print(f"\n{'='*80}")
print("LOADING COMPLETE")
print(f"{'='*80}")

# Check what was loaded
for commodity, accuracy_results in all_results.items():
    print(f"\n{commodity.upper()}: Loaded {len(accuracy_results)} accuracy levels")

## Analysis 1: Performance vs Accuracy Curves

In [ ]:
for COMMODITY in COMMODITY_CONFIGS.keys():
    if COMMODITY not in all_results or len(all_results[COMMODITY]) == 0:
        print(f"\n⚠️  No results for {COMMODITY.upper()} - skipping")
        continue
    
    print(f"\n{'='*80}")
    print(f"PERFORMANCE VS ACCURACY ANALYSIS: {COMMODITY.upper()}")
    print(f"{'='*80}")
    
    commodity_results = all_results[COMMODITY]
    
    # Collect data for visualization
    accuracy_data = []
    
    for accuracy in sorted(commodity_results.keys()):
        summary_df = commodity_results[accuracy]['summary']
        
        for _, row in summary_df.iterrows():
            accuracy_data.append({
                'accuracy': accuracy,
                'strategy': row['strategy'],
                'type': row['type'],
                'net_earnings': row['net_earnings'],
                'total_revenue': row['total_revenue'],
                'avg_sale_price': row['avg_sale_price'],
                'n_trades': row['n_trades'],
                'total_costs': row['total_costs']
            })
    
    accuracy_df = pd.DataFrame(accuracy_data)
    
    # Create comprehensive comparison figure
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle(f'{COMMODITY.title()} - Performance vs Prediction Accuracy', 
                 fontsize=18, fontweight='bold', y=0.995)
    
    # Plot 1: Net Earnings vs Accuracy (All Strategies)
    ax1 = axes[0, 0]
    for strategy in accuracy_df['strategy'].unique():
        strat_data = accuracy_df[accuracy_df['strategy'] == strategy]
        is_prediction = strategy in PREDICTION_STRATEGIES
        
        ax1.plot(strat_data['accuracy'], strat_data['net_earnings'],
                marker='o', linewidth=2 if is_prediction else 1,
                linestyle='-' if is_prediction else '--',
                alpha=0.9 if is_prediction else 0.6,
                label=strategy)
    
    ax1.set_xlabel('Prediction Accuracy (%)', fontweight='bold')
    ax1.set_ylabel('Net Earnings ($)', fontweight='bold')
    ax1.set_title('Net Earnings vs Accuracy', fontweight='bold')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    ax1.grid(True, alpha=0.3)
    ax1.set_xticks(SYNTHETIC_ACCURACIES)
    
    # Plot 2: Prediction Strategies Only (Cleaner View)
    ax2 = axes[0, 1]
    pred_data = accuracy_df[accuracy_df['strategy'].isin(PREDICTION_STRATEGIES)]
    
    for strategy in PREDICTION_STRATEGIES:
        strat_data = pred_data[pred_data['strategy'] == strategy]
        ax2.plot(strat_data['accuracy'], strat_data['net_earnings'],
                marker='o', linewidth=2.5, label=strategy)
    
    ax2.set_xlabel('Prediction Accuracy (%)', fontweight='bold')
    ax2.set_ylabel('Net Earnings ($)', fontweight='bold')
    ax2.set_title('Prediction Strategies Only', fontweight='bold')
    ax2.legend(fontsize=9)
    ax2.grid(True, alpha=0.3)
    ax2.set_xticks(SYNTHETIC_ACCURACIES)
    
    # Plot 3: Best Prediction vs Best Baseline
    ax3 = axes[0, 2]
    
    best_by_accuracy = []
    for accuracy in SYNTHETIC_ACCURACIES:
        acc_data = accuracy_df[accuracy_df['accuracy'] == accuracy]
        
        baseline_data = acc_data[acc_data['strategy'].isin(BASELINE_STRATEGIES)]
        pred_data_local = acc_data[acc_data['strategy'].isin(PREDICTION_STRATEGIES)]
        
        best_baseline = baseline_data['net_earnings'].max() if len(baseline_data) > 0 else 0
        best_prediction = pred_data_local['net_earnings'].max() if len(pred_data_local) > 0 else 0
        
        best_by_accuracy.append({
            'accuracy': accuracy,
            'best_baseline': best_baseline,
            'best_prediction': best_prediction,
            'advantage': best_prediction - best_baseline
        })
    
    best_df = pd.DataFrame(best_by_accuracy)
    
    ax3.plot(best_df['accuracy'], best_df['best_baseline'],
            marker='s', linewidth=2.5, label='Best Baseline', color='steelblue')
    ax3.plot(best_df['accuracy'], best_df['best_prediction'],
            marker='o', linewidth=2.5, label='Best Prediction', color='orangered')
    
    ax3.set_xlabel('Prediction Accuracy (%)', fontweight='bold')
    ax3.set_ylabel('Net Earnings ($)', fontweight='bold')
    ax3.set_title('Best Strategy per Category', fontweight='bold')
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3)
    ax3.set_xticks(SYNTHETIC_ACCURACIES)
    
    # Plot 4: Prediction Advantage ($)
    ax4 = axes[1, 0]
    
    colors = [ACCURACY_COLORS[acc] for acc in best_df['accuracy']]
    bars = ax4.bar(best_df['accuracy'], best_df['advantage'], 
                   color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    
    # Add value labels
    for bar, val in zip(bars, best_df['advantage']):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'${val:,.0f}',
                ha='center', va='bottom' if val > 0 else 'top',
                fontweight='bold', fontsize=11)
    
    ax4.axhline(y=0, color='black', linestyle='-', linewidth=1)
    ax4.set_xlabel('Prediction Accuracy (%)', fontweight='bold')
    ax4.set_ylabel('Prediction Advantage ($)', fontweight='bold')
    ax4.set_title('Dollar Advantage of Predictions', fontweight='bold')
    ax4.grid(True, alpha=0.3, axis='y')
    ax4.set_xticks(SYNTHETIC_ACCURACIES)
    
    # Plot 5: Prediction Advantage (%)
    ax5 = axes[1, 1]
    
    pct_advantage = (best_df['advantage'] / best_df['best_baseline']) * 100
    bars = ax5.bar(best_df['accuracy'], pct_advantage,
                   color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    
    # Add value labels
    for bar, val in zip(bars, pct_advantage):
        height = bar.get_height()
        ax5.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:+.1f}%',
                ha='center', va='bottom' if val > 0 else 'top',
                fontweight='bold', fontsize=11)
    
    ax5.axhline(y=0, color='black', linestyle='-', linewidth=1)
    ax5.set_xlabel('Prediction Accuracy (%)', fontweight='bold')
    ax5.set_ylabel('Prediction Advantage (%)', fontweight='bold')
    ax5.set_title('Percentage Improvement with Predictions', fontweight='bold')
    ax5.grid(True, alpha=0.3, axis='y')
    ax5.set_xticks(SYNTHETIC_ACCURACIES)
    
    # Plot 6: Summary Table
    ax6 = axes[1, 2]
    ax6.axis('off')
    
    # Create summary data
    table_data = []
    for _, row in best_df.iterrows():
        pct = ((row['best_prediction'] - row['best_baseline']) / row['best_baseline']) * 100
        table_data.append([
            f"{row['accuracy']}%",
            f"${row['best_baseline']:,.0f}",
            f"${row['best_prediction']:,.0f}",
            f"${row['advantage']:+,.0f}",
            f"{pct:+.1f}%"
        ])
    
    table = ax6.table(cellText=table_data,
                     colLabels=['Accuracy', 'Best\nBaseline', 'Best\nPrediction', 
                               'Advantage\n($)', 'Advantage\n(%)'],
                     cellLoc='center',
                     loc='center',
                     colWidths=[0.15, 0.20, 0.20, 0.20, 0.15])
    
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 3)
    
    # Style header
    for i in range(5):
        table[(0, i)].set_facecolor('#2c3e50')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    # Style rows with accuracy colors
    for i, acc in enumerate(SYNTHETIC_ACCURACIES, 1):
        for j in range(5):
            table[(i, j)].set_facecolor(ACCURACY_COLORS[acc])
            table[(i, j)].set_alpha(0.3)
    
    ax6.set_title('Summary Table', fontsize=13, fontweight='bold', pad=20)
    
    plt.tight_layout()
    
    # Save figure
    output_path = f'{VOLUME_PATH}/synthetic_accuracy_comparison_{COMMODITY}.png'
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"\n✓ Saved: {output_path}")
    
    plt.show()
    plt.close()
    
    # Save data table
    csv_path = f'{VOLUME_PATH}/synthetic_accuracy_comparison_{COMMODITY}.csv'
    best_df.to_csv(csv_path, index=False)
    print(f"✓ Saved: {csv_path}")

## Analysis 2: Monotonicity Check

In [ ]:
print("\n" + "="*80)
print("MONOTONICITY VALIDATION")
print("="*80)
print("\nChecking if performance improves monotonically with accuracy...")
print("(This should be TRUE for prediction strategies if logic is correct)\n")

for COMMODITY in all_results.keys():
    if len(all_results[COMMODITY]) == 0:
        continue
    
    print(f"\n{'─'*80}")
    print(f"{COMMODITY.upper()}")
    print(f"{'─'*80}")
    
    commodity_results = all_results[COMMODITY]
    
    # Check each prediction strategy
    for strategy in PREDICTION_STRATEGIES:
        earnings_by_acc = {}
        
        for accuracy in sorted(commodity_results.keys()):
            summary_df = commodity_results[accuracy]['summary']
            strat_row = summary_df[summary_df['strategy'] == strategy]
            
            if len(strat_row) > 0:
                earnings_by_acc[accuracy] = strat_row['net_earnings'].iloc[0]
        
        if len(earnings_by_acc) < 2:
            continue
        
        # Check monotonicity
        accuracies = sorted(earnings_by_acc.keys())
        earnings = [earnings_by_acc[acc] for acc in accuracies]
        
        is_monotonic = all(earnings[i] <= earnings[i+1] for i in range(len(earnings)-1))
        
        print(f"\n  {strategy}:")
        for acc in accuracies:
            print(f"    {acc}%: ${earnings_by_acc[acc]:,.2f}")
        
        if is_monotonic:
            print(f"    ✓ MONOTONIC: Performance improves with accuracy")
        else:
            print(f"    ✗ NOT MONOTONIC: Performance does NOT consistently improve")
            print(f"    ⚠️  WARNING: This may indicate a logic issue!")
        
        # Calculate correlation
        if len(accuracies) >= 3:
            corr, p_value = stats.pearsonr(accuracies, earnings)
            print(f"    Correlation: {corr:.3f} (p={p_value:.4f})")
            
            if corr > 0.9 and p_value < 0.05:
                print(f"    ✓ Strong positive correlation (expected)")
            elif corr < 0.5:
                print(f"    ⚠️  Weak correlation - predictions may not be used effectively")

## Analysis 3: 90% Accuracy Validation

In [ ]:
print("\n" + "="*80)
print("90% ACCURACY VALIDATION TEST")
print("="*80)
print("\nCRITICAL CHECK: At 90% accuracy, prediction strategies MUST beat baselines.")
print("If not, there's a bug in the trading logic.\n")

validation_results = []

for COMMODITY in all_results.keys():
    if len(all_results[COMMODITY]) == 0 or 90 not in all_results[COMMODITY]:
        print(f"\n⚠️  No 90% accuracy results for {COMMODITY.upper()}")
        continue
    
    print(f"\n{'─'*80}")
    print(f"{COMMODITY.upper()} - 90% ACCURACY")
    print(f"{'─'*80}")
    
    summary_df = all_results[COMMODITY][90]['summary']
    
    # Get best baseline
    baseline_df = summary_df[summary_df['strategy'].isin(BASELINE_STRATEGIES)]
    best_baseline = baseline_df.loc[baseline_df['net_earnings'].idxmax()]
    
    print(f"\nBest Baseline: {best_baseline['strategy']}")
    print(f"  Net Earnings: ${best_baseline['net_earnings']:,.2f}")
    
    print(f"\nPrediction Strategies:")
    
    pred_df = summary_df[summary_df['strategy'].isin(PREDICTION_STRATEGIES)]
    
    all_pass = True
    
    for _, pred_row in pred_df.iterrows():
        advantage = pred_row['net_earnings'] - best_baseline['net_earnings']
        pct_advantage = (advantage / best_baseline['net_earnings']) * 100
        
        beats_baseline = advantage > 0
        
        status = "✓" if beats_baseline else "✗"
        
        print(f"\n  {status} {pred_row['strategy']}:")
        print(f"      Net Earnings: ${pred_row['net_earnings']:,.2f}")
        print(f"      Advantage: ${advantage:+,.2f} ({pct_advantage:+.1f}%)")
        
        if not beats_baseline:
            all_pass = False
            print(f"      ⚠️  FAILS: Does not beat best baseline!")
        else:
            if pct_advantage > 10:
                print(f"      ✓ STRONG: {pct_advantage:.1f}% improvement")
            elif pct_advantage > 5:
                print(f"      ✓ MODERATE: {pct_advantage:.1f}% improvement")
            else:
                print(f"      ⚠️  WEAK: Only {pct_advantage:.1f}% improvement")
        
        validation_results.append({
            'commodity': COMMODITY,
            'strategy': pred_row['strategy'],
            'net_earnings': pred_row['net_earnings'],
            'baseline_earnings': best_baseline['net_earnings'],
            'advantage': advantage,
            'pct_advantage': pct_advantage,
            'beats_baseline': beats_baseline
        })
    
    print(f"\n{'─'*40}")
    if all_pass:
        print(f"✓ PASS: All prediction strategies beat baseline at 90% accuracy")
    else:
        print(f"✗ FAIL: Some prediction strategies don't beat baseline!")
        print(f"⚠️  CRITICAL: This indicates a bug in the trading logic.")

# Summary table
if len(validation_results) > 0:
    validation_df = pd.DataFrame(validation_results)
    
    print(f"\n\n{'='*80}")
    print("VALIDATION SUMMARY")
    print(f"{'='*80}\n")
    
    pass_count = validation_df['beats_baseline'].sum()
    total_count = len(validation_df)
    pass_rate = (pass_count / total_count) * 100
    
    print(f"Results: {pass_count}/{total_count} prediction strategies beat baseline ({pass_rate:.0f}%)")
    
    if pass_rate == 100:
        print(f"\n✓✓✓ EXCELLENT: All prediction strategies work as expected!")
    elif pass_rate >= 80:
        print(f"\n✓ GOOD: Most prediction strategies work, but investigate failures")
    elif pass_rate >= 50:
        print(f"\n⚠️  CONCERN: Many prediction strategies fail - review logic")
    else:
        print(f"\n✗✗✗ CRITICAL: Trading logic appears broken - debug required!")
    
    # Save validation results
    val_csv = f'{VOLUME_PATH}/validation_90pct_accuracy.csv'
    validation_df.to_csv(val_csv, index=False)
    print(f"\n✓ Saved validation results: {val_csv}")

## Analysis 4: Accuracy Threshold Identification

In [ ]:
print("\n" + "="*80)
print("ACCURACY THRESHOLD ANALYSIS")
print("="*80)
print("\nAt what accuracy do predictions become valuable?\n")

for COMMODITY in all_results.keys():
    if len(all_results[COMMODITY]) == 0:
        continue
    
    print(f"\n{'─'*80}")
    print(f"{COMMODITY.upper()}")
    print(f"{'─'*80}")
    
    commodity_results = all_results[COMMODITY]
    
    # For each prediction strategy, find break-even accuracy
    for strategy in PREDICTION_STRATEGIES:
        print(f"\n  {strategy}:")
        
        threshold_found = None
        
        for accuracy in sorted(commodity_results.keys()):
            summary_df = commodity_results[accuracy]['summary']
            
            # Get strategy earnings
            strat_row = summary_df[summary_df['strategy'] == strategy]
            if len(strat_row) == 0:
                continue
            
            strat_earnings = strat_row['net_earnings'].iloc[0]
            
            # Get best baseline
            baseline_df = summary_df[summary_df['strategy'].isin(BASELINE_STRATEGIES)]
            best_baseline_earnings = baseline_df['net_earnings'].max()
            
            beats_baseline = strat_earnings > best_baseline_earnings
            advantage = strat_earnings - best_baseline_earnings
            
            print(f"    {accuracy}%: ${strat_earnings:,.0f} (baseline: ${best_baseline_earnings:,.0f}) → "
                  f"{'+' if beats_baseline else '-'}${abs(advantage):,.0f}")
            
            if beats_baseline and threshold_found is None:
                threshold_found = accuracy
        
        if threshold_found:
            print(f"    ✓ Threshold: {threshold_found}% (first accuracy where predictions beat baseline)")
        else:
            print(f"    ✗ No threshold found (predictions never beat baseline in tested range)")

## Analysis 5: Cross-Commodity Comparison

In [ ]:
print("\n" + "="*80)
print("CROSS-COMMODITY COMPARISON")
print("="*80)

# Create comparison table
comparison_data = []

for COMMODITY in all_results.keys():
    commodity_results = all_results[COMMODITY]
    
    for accuracy in sorted(commodity_results.keys()):
        summary_df = commodity_results[accuracy]['summary']
        
        baseline_df = summary_df[summary_df['strategy'].isin(BASELINE_STRATEGIES)]
        pred_df = summary_df[summary_df['strategy'].isin(PREDICTION_STRATEGIES)]
        
        best_baseline = baseline_df['net_earnings'].max() if len(baseline_df) > 0 else 0
        best_prediction = pred_df['net_earnings'].max() if len(pred_df) > 0 else 0
        
        comparison_data.append({
            'commodity': COMMODITY,
            'accuracy': accuracy,
            'best_baseline': best_baseline,
            'best_prediction': best_prediction,
            'advantage': best_prediction - best_baseline,
            'pct_advantage': ((best_prediction - best_baseline) / best_baseline * 100) if best_baseline > 0 else 0
        })

if len(comparison_data) > 0:
    comparison_df = pd.DataFrame(comparison_data)
    
    # Pivot for easier viewing
    pivot_pct = comparison_df.pivot(index='commodity', columns='accuracy', values='pct_advantage')
    
    print("\nPercentage Advantage of Predictions by Commodity and Accuracy:\n")
    print(pivot_pct.to_string())
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Dollar advantage
    ax1 = axes[0]
    for commodity in comparison_df['commodity'].unique():
        comm_data = comparison_df[comparison_df['commodity'] == commodity]
        ax1.plot(comm_data['accuracy'], comm_data['advantage'],
                marker='o', linewidth=2.5, label=commodity.title())
    
    ax1.axhline(y=0, color='black', linestyle='--', linewidth=1)
    ax1.set_xlabel('Prediction Accuracy (%)', fontweight='bold', fontsize=12)
    ax1.set_ylabel('Dollar Advantage ($)', fontweight='bold', fontsize=12)
    ax1.set_title('Prediction Advantage by Commodity', fontweight='bold', fontsize=14)
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    ax1.set_xticks(SYNTHETIC_ACCURACIES)
    
    # Plot 2: Percentage advantage
    ax2 = axes[1]
    for commodity in comparison_df['commodity'].unique():
        comm_data = comparison_df[comparison_df['commodity'] == commodity]
        ax2.plot(comm_data['accuracy'], comm_data['pct_advantage'],
                marker='o', linewidth=2.5, label=commodity.title())
    
    ax2.axhline(y=0, color='black', linestyle='--', linewidth=1)
    ax2.set_xlabel('Prediction Accuracy (%)', fontweight='bold', fontsize=12)
    ax2.set_ylabel('Percentage Advantage (%)', fontweight='bold', fontsize=12)
    ax2.set_title('Percentage Improvement by Commodity', fontweight='bold', fontsize=14)
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    ax2.set_xticks(SYNTHETIC_ACCURACIES)
    
    plt.tight_layout()
    
    cross_path = f'{VOLUME_PATH}/cross_commodity_accuracy_comparison.png'
    plt.savefig(cross_path, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"\n✓ Saved: {cross_path}")
    
    plt.show()
    plt.close()
    
    # Save comparison table
    cross_csv = f'{VOLUME_PATH}/cross_commodity_accuracy_comparison.csv'
    comparison_df.to_csv(cross_csv, index=False)
    print(f"✓ Saved: {cross_csv}")

## Final Summary

In [ ]:
print("\n" + "="*80)
print("SYNTHETIC ACCURACY ANALYSIS - FINAL SUMMARY")
print("="*80)

print("\n📊 ANALYSIS COMPLETE")
print("\nKey Questions Answered:")
print("  1. ✓ Performance vs Accuracy curves generated")
print("  2. ✓ Monotonicity validated (should improve with accuracy)")
print("  3. ✓ 90% accuracy validation performed (critical test)")
print("  4. ✓ Accuracy threshold identified (where predictions become valuable)")
print("  5. ✓ Cross-commodity comparison completed")

print("\n📁 Outputs Saved:")
print(f"  • Accuracy comparison charts (per commodity)")
print(f"  • Validation results (90% accuracy test)")
print(f"  • Cross-commodity comparison")
print(f"  • CSV data tables")

print("\n⚠️  CRITICAL VALIDATION:")
print("  If 90% accuracy predictions DON'T show advantage:")
print("    → Trading logic has bugs")
print("    → Review strategy implementations")
print("    → Check cost assumptions")
print("    → Verify data isn't leaking future information")

print("\n✓ Notebook 11 Complete")
print("="*80)